In [ ]:
import mlagents
from mlagents_envs.environment import UnityEnvironment as UE
import numpy as np
from mlagents_envs.environment import ActionTuple


In [ ]:
env =  UE(file_name="280623_1\\build",seed=1,side_channels=[],worker_id=0,no_graphics = False)
env.reset()
behavior_names = list(env.behavior_specs.keys())
behavior_value = list(env.behavior_specs.values())
DecisionSteps, TerminalSteps = env.get_steps(behavior_names[0])
agentsNum = len(DecisionSteps.agent_id)

In [ ]:
import torch
import torch.nn as nn
#(3,128,128) --> (64,7,7)
image = torch.tensor(DecisionSteps.obs[0]).reshape(1,3,128,128)
image.size()

#  Model

In [ ]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F

vision_output_dim = 3136
num_words = 35  # Number of unique words in the vocabulary
language_output_dim = 128
embedding_dim = 128
mixing_dim = 256
lstm_hidden_dim = 256
num_actions = 4

# (3,128,128) --> (64,7,7) = 3136 (3-layer CNN)
class VisualModule(nn.Module): 
    def __init__(self):
        super(VisualModule, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=3, padding=0),
            nn.ReLU()
        )
        # self.conv = nn.Sequential(
        #     nn.Conv2d(3, 32, kernel_size=5, stride=2, padding=2),
        #     nn.ReLU(),
        #     nn.Conv2d(32, 64, kernel_size=5, stride=2, padding=2),
        #     nn.ReLU(),
        #     nn.Conv2d(64, 128, kernel_size=5, stride=2, padding=2),
        #     nn.ReLU(),
        #     nn.Conv2d(128, 64, kernel_size=5, stride=2, padding=1),
        #     nn.ReLU(),
        #     nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
        #     nn.ReLU(),
        #     nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
        #     nn.ReLU(),
        # )

    def forward(self, vt):
        encoded_vt = self.conv(vt)
        return encoded_vt.view(vt.size(0), -1).squeeze()

# one-hot encoding [0 0 1 0 0] --> 128 dimensional embedding (FF)
# S1:5 S2:5 S3:11 S4:9 --> 30 + 5 (noun) = 35 in total
class LanguageModule(nn.Module): 
    def __init__(self, num_words, embedding_dim):
        super(LanguageModule, self).__init__()
        self.embedding = nn.Linear(num_words, embedding_dim)

    def forward(self, lt):
        embedded_lt = self.embedding(lt)
        return embedded_lt

# 3136(vision) + 128 (language) --> 256 dimensional embedding (FF)
class MixingModule(nn.Module):
    def __init__(self, vision_output_dim, language_output_dim, mixing_dim):
        super(MixingModule, self).__init__()
        self.linear = nn.Linear(vision_output_dim + language_output_dim, mixing_dim)

    def forward(self, vision_output, language_output):
        combined_output = torch.cat((vision_output, language_output), dim=0)
        mixed_output = self.linear(combined_output)
        return mixed_output

class LSTMModule(nn.Module):
    def __init__(self,mixing_dim,lstm_hidden_dim):
        super(LSTMModule, self).__init__()
        self.lstm = nn.LSTMCell(mixing_dim, lstm_hidden_dim)
    
    def forward(self,mixed_output,lstm_hidden_state):
        lstm_hidden_state = self.lstm(mixed_output, lstm_hidden_state) 
        # lstm_output = lstm_hidden_state[0] # output is (hidden_state,cell_state), we need hidden state, shape (1,256)
        return lstm_hidden_state

class Agent(nn.Module):
    def __init__(self, num_words, embedding_dim, vision_output_dim, language_output_dim, mixing_dim, lstm_hidden_dim,num_actions):
        super(Agent, self).__init__()
        self.language_module = LanguageModule(num_words, embedding_dim)
        self.visual_module = VisualModule()
        self.mixing_module = MixingModule(vision_output_dim, language_output_dim, mixing_dim)
        self.lstm_module = LSTMModule(mixing_dim, lstm_hidden_dim)
        self.action_predictor = nn.Linear(lstm_hidden_dim, num_actions)
        self.value_estimator = nn.Linear(lstm_hidden_dim, 1)

    def forward(self, vt, lt, lstm_hidden_state):
        vision_output = self.visual_module(vt)
        language_output = self.language_module(lt)
        mixed_output = self.mixing_module(vision_output, language_output).unsqueeze(0)
        lstm_output = self.lstm_module(mixed_output,lstm_hidden_state)
        action_probs = self.action_predictor(lstm_output[0]) 
        value_estimate = self.value_estimator(lstm_output[0])
        return action_probs,value_estimate,lstm_output
        
        
    def save(self, episode, ALG_NAME, ENV_ID):
        path = os.path.join('model', '_'.join([ALG_NAME, ENV_ID]))
        if not os.path.exists(path):
            os.makedirs(path)
        torch.save(self.state_dict(), os.path.join(path, f'agent_{episode}.pt'))

    def load(self, episode, ALG_NAME, ENV_ID):
        path = os.path.join('model', '_'.join([ALG_NAME, ENV_ID]))
        self.load_state_dict(torch.load(os.path.join(path, f'agent_{episode}.pt')))    

        


In [ ]:
visual_model = VisualModule()
vt = image
image_emb = visual_model(vt)
print(f'The shape of image embedding is {image_emb.size()}')

index = 5
language_model = LanguageModule(num_words,embedding_dim)
lt = torch.eye(num_words)[:, index]
language_emb = language_model(lt)
print(f'The shape of language embedding is {language_emb.size()}')

mixing_model = MixingModule(vision_output_dim,language_output_dim,mixing_dim)
mix_emb = mixing_model(image_emb,language_emb)
print(f'The shape of mix embedding is {mix_emb.size()}')

lstm = LSTMModule(mixing_dim,lstm_hidden_dim)
lstm_hidden_state = (torch.zeros(1, lstm_hidden_dim), torch.zeros(1, lstm_hidden_dim))
hidden_state = lstm(mix_emb.unsqueeze(0),lstm_hidden_state)
print(f'The shape of lstm hidden state is {hidden_state[0].size()}')

agent = Agent(num_words, embedding_dim, vision_output_dim, language_output_dim, mixing_dim, lstm_hidden_dim,num_actions)
action, value, lstm_hidden_state= agent(vt,lt,lstm_hidden_state)


# Train

In [ ]:
import argparse
import time
import json
import matplotlib.pyplot as plt
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions import Categorical
train = True
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device) 
ALG_NAME = 'S0'
ENV_ID = '1'
TRAIN_EPISODES = 10000  # number of overall episodes for training
TEST_EPISODES = 10  # number of overall episodes for testing
MAX_STEPS = 500  # maximum time step in one episode
LAM = 0.95  # reward discount in TD error
lr = 0.0001  #0.00005 
speed = 1
num_steps = 100 # the step for updating the network

if __name__ == '__main__':
    agent = Agent(num_words, embedding_dim, vision_output_dim, language_output_dim, mixing_dim, lstm_hidden_dim,num_actions)
    agent.to(device)
    optimizer = optim.RMSprop(agent.parameters(), lr=lr)
    best_score = 0
    if train:
        entropy_term = 0
        all_episode_reward = []
        all_average_reward = []
        all_steps = []
        all_actor_loss = []
        all_critic_loss = []
        all_entropy_loss = []
        all_total_loss = []
        tracked_agent = -1
        for episode in range(TRAIN_EPISODES):
            t0 = time.time()
            episode_reward = 0
            # env.reset()
            behavior_name=list(env.behavior_specs)[0]
            spec=env.behavior_specs[behavior_name]
            # state = env.reset().astype(np.float32)
            STEPS = 0

            # state -- vt, lt, lstm
            vt = torch.tensor(DecisionSteps.obs[0]).reshape(1,3,128,128).to(device)
            index = 5 #e.g
            lt = torch.eye(num_words)[:, index].to(device)
            lstm_hidden_state = (torch.zeros(1, lstm_hidden_dim).to(device), torch.zeros(1, lstm_hidden_dim).to(device))
            done = False
            while True:

                # Need to use when calculating the loss
                log_probs = []
                # values = []
                values = torch.empty(0).to(device)
                rewards = []

                for steps in range(num_steps):
                    lstm_hidden_state = tuple(tensor.detach() for tensor in lstm_hidden_state)
                    STEPS += 1
                    policy_dist, value, lstm_hidden_state = agent(vt,lt,lstm_hidden_state)
                    # value = value.detach()
                    dist = F.softmax(policy_dist.detach(),dim=1).cpu().numpy()
                    

                    action_dist = Categorical(F.softmax(policy_dist.detach(),dim=1))
                    # action_dist = Categorical(F.softmax(policy_dist,dim=1))
                    action = action_dist.sample() # sample an action from action_dist
                    action_onehot = F.one_hot(torch.tensor(action),num_actions).cpu()
                    
                    log_prob = torch.log(F.softmax(policy_dist,dim=1)[0][action])
                    # log_prob = torch.log(F.softmax(policy_dist,dim=1)[0][action])
                    # entropy = -np.sum(np.mean(dist)* np.log(dist))
                    entropy = F.cross_entropy(policy_dist.detach(), action)

                    discrete_actions = np.array(action_onehot).reshape(1,4)*speed
                    action_tuple = ActionTuple()
                    action_tuple.add_discrete(discrete_actions)
                    env.set_actions(behavior_name,action_tuple)
                    env.step()
                    decision_steps, terminal_steps = env.get_steps(behavior_name)

                    if tracked_agent == -1 and len(decision_steps) >= 1:
                        tracked_agent = decision_steps.agent_id[0]
                        # print(tracked_agent)

                    if tracked_agent in terminal_steps: # roll over or hit the target
                        print('Agent in terminal steps')
                        done = True
                        reward = terminal_steps[tracked_agent].reward
                        if reward > 0:
                            pass
                        else: reward = -1 # roll over or other unseen conditions

                        print(f'Terminal Step reward: {reward}')

                    elif tracked_agent in decision_steps: # the agent which requires action
                        reward = decision_steps[tracked_agent].reward
                        # print(f'Decision Step reward: {reward}')
                        if reward<0:
                            print(f'Decision Step reward: {reward}')
                    if STEPS >= MAX_STEPS:
                        reward = -10
                        print(f'Max Step Reward: {reward}')
                        env.reset()
                        done = True
                    if STEPS % 100 == 0:
                        print (f'Step: {STEPS}')

                    episode_reward = episode_reward + reward

                    rewards.append(reward)
                    # values.append(value)
                    values = torch.cat((values, value), dim=0)
                    log_probs.append(log_prob)
                    entropy_term = entropy_term + entropy
                    vt_new = torch.tensor(decision_steps.obs[0]).reshape(1,3,128,128).to(device)
                    vt = vt_new

                    if done or steps == num_steps-1:
                        break
                
                
                discounted_rewards = np.zeros_like(values.cpu().detach().numpy())
                cumulative = 0
                for t in reversed(range(len(rewards))):
                    cumulative = rewards[t] + LAM * cumulative # Monte Carlo
                    discounted_rewards[t] = cumulative
                # print(f'rewards:{rewards}, discounted_rewards:{discounted_rewards}')
                # Advantage Actor Critic

                # Qvals[-1] = rewards[t] + LAM * Qval      or       Qvals[-1] = rewards[t]                   
                # for t in range(len(rewards)-1):
                #         Qvals[t] = rewards[t] + LAM * values[t+1]
                
                # r_(t+1) = R(s_t|a_t)--> reward[t]        a_t, V_t = agent(s_t)
                # A_t = r_(t+1) + LAM * V_(t+1) - V_t 
                #     = Q_t - V_t
                
                # Monte Carlo Advantage = reward + LAM * cumulative_reward
                # Actor_loss = -log(pai(s_t|a_t))*A_t
                # Critic_loss = A_t.pow(2) *0.5
                # Entropy_loss = -F.entropy(pai(St),index) * 0.001

                # entropy = -np.sum(np.mean(dist) * np.log(dist))
                
                #update actor critic
                
                # values = torch.FloatTensor(values).requires_grad_(True).to(device)
                discounted_rewards = torch.FloatTensor(discounted_rewards.astype(np.float32)).to(device)
                log_probs = torch.stack(log_probs)
                advantage = discounted_rewards - values
                actor_loss = (-log_probs * advantage).mean()
                critic_loss = 0.5 * torch.square(advantage).mean()
                entropy_term /= num_steps
                entropy_loss = -0.001 * entropy_term
                ac_loss = actor_loss + critic_loss + entropy_loss
                optimizer.zero_grad()
                ac_loss.backward()
                optimizer.step()
                # for name, param in agent.named_parameters():
                #     if param.grad is not None:
                #         print(name, param.grad)
                #     else:
                #         print(name, "gradients not computed")
                # for name, param in agent.named_parameters():
                #     if name == 'value_estimator.weight':
                #         print(name, param)
                
                
                if done: break


            all_episode_reward.append(float(episode_reward))
            all_steps.append(STEPS)
            all_actor_loss.append(float(actor_loss))
            all_critic_loss.append(float(critic_loss))
            all_entropy_loss.append(float(entropy_loss))
            all_total_loss.append(float(ac_loss))
            if episode >= 100:
                avg_score = np.mean(all_episode_reward[-100:])
                all_average_reward.append(avg_score)
                if avg_score > best_score:
                    best_score = avg_score
                    agent.save(episode, ALG_NAME, ENV_ID)
                    print(f'-----The best score for averaging previous 100 episode reward is {best_score}. Model has been saved-----')
                print('Training  | Episode: {}/{}  | Episode Reward: {:.0f}  | Average Reward {:.2f}  | Actor loss: {:.2f} | Critic loss: {:.2f} | Entropy loss: {:.4f}  | Total Loss: {:.2f} | Total Steps: {}' \
                    .format(episode + 1, TRAIN_EPISODES, episode_reward, avg_score, actor_loss, critic_loss,entropy_loss,  ac_loss, STEPS))
            else:  print('Training  | Episode: {}/{}  | Episode Reward: {:.0f}  | Actor loss: {:.2f} | Critic loss: {:.2f} | Entropy loss: {:.4f}  | Total Loss: {:.2f} | Total Steps: {}' \
                    .format(episode + 1, TRAIN_EPISODES, episode_reward, actor_loss, critic_loss, entropy_loss,  ac_loss, STEPS))
            if episode%500 == 0:
                    agent.save(episode, ALG_NAME, ENV_ID)
                    print("Model has been saved")
        print(all_average_reward)
        agent.save(episode ,ALG_NAME, ENV_ID)
        print("Model has been saved")

        data = {
                    'all_average_reward': all_average_reward,
                    'all_episode_reward': all_episode_reward,
                    'all_actor_loss': all_actor_loss,
                    'all_critic_loss': all_critic_loss,
                    'all_entropy_loss': all_entropy_loss,
                    'all_total_loss': all_total_loss,
                    'all_steps': all_steps,
                } 
        file_path = f'{ALG_NAME}_{ENV_ID}.txt'
        with open(file_path, 'w') as file:
            json.dump(data, file)



# Test

In [ ]:
from torch.distributions import Categorical
device = torch.device("cpu")
episode = 9190
speed = 1
MAX_STEPS = 500
TEST_EPISODES = 100
ALG_NAME = 'S0_with'
ENV_ID = '3'
tracked_agent = -1
env.reset()
agent = Agent(num_words, embedding_dim, vision_output_dim, language_output_dim, mixing_dim, lstm_hidden_dim,num_actions)
agent.load(episode,ALG_NAME,ENV_ID)
average = 0
for episode in range(TEST_EPISODES):
            STEPS = 0
            episode_reward = 0
            # env.reset()
            behavior_name=list(env.behavior_specs)[0]
            spec=env.behavior_specs[behavior_name]
            # state = env.reset().astype(np.float32)

            # state -- vt, lt, lstm
            vt = torch.tensor(DecisionSteps.obs[0]).reshape(1,3,128,128).to(device)
            index = 5 #e.g
            lt = torch.eye(num_words)[:, index].to(device)
            lstm_hidden_state = (torch.zeros(1, lstm_hidden_dim).to(device), torch.zeros(1, lstm_hidden_dim).to(device))
            done = False
            while not done:
                STEPS += 1                
                lstm_hidden_state = tuple(tensor.detach() for tensor in lstm_hidden_state)
                policy_dist, value, lstm_hidden_state = agent(vt,lt,lstm_hidden_state)
                # value = value.detach()
                dist = F.softmax(policy_dist.detach(),dim=1).cpu().numpy()
                

                action_dist = Categorical(F.softmax(policy_dist.detach(),dim=1))
                # action_dist = Categorical(F.softmax(policy_dist,dim=1))
                action = action_dist.sample() # sample an action from action_dist
                action_onehot = F.one_hot(torch.tensor(action),num_actions).cpu()

                discrete_actions = np.array(action_onehot).reshape(1,4)*speed
                action_tuple = ActionTuple()
                action_tuple.add_discrete(discrete_actions)
                env.set_actions(behavior_name,action_tuple)
                env.step()
                decision_steps, terminal_steps = env.get_steps(behavior_name)

                if tracked_agent == -1 and len(decision_steps) >= 1:
                    tracked_agent = decision_steps.agent_id[0]
                    # print(tracked_agent)

                if tracked_agent in terminal_steps: # roll over or hit the target
                    print('Agent in terminal steps')
                    done = True
                    reward = terminal_steps[tracked_agent].reward
                    if reward > 0:
                        pass
                    else: reward = -1 # roll over or other unseen conditions

                    print(f'Terminal Step reward: {reward}')

                elif tracked_agent in decision_steps: # the agent which requires action
                    reward = decision_steps[tracked_agent].reward
                    # print(f'Decision Step reward: {reward}')
                    # if reward<0:
                    #     print(f'Decision Step reward: {reward}')

                if STEPS >= MAX_STEPS:
                        reward = -10
                        print(f'Max Step Reward: {reward}')
                        env.reset()
                        done = True

                episode_reward = episode_reward + reward
                vt_new = torch.tensor(decision_steps.obs[0]).reshape(1,3,128,128).to(device)
                vt = vt_new
            average += episode_reward / TEST_EPISODES
            print(f'Episode: {episode}, Episode reward: {episode_reward}')
print(f'Average Episode Reward: {average}')


# Random Action

In [ ]:
# Try to find the optimized MAX_STEP
# speed 1: 10000+ speed 2:1822 speed 3: 1918 speed 4: 900 speed 5: 951 speed 6:964 speed 7: 1181 
import random
import torch
import torch.nn.functional as F
# speed = 1
TRAIN_EPISODES = 20
tracked_agent = -1
num_actions = 4
average = 0
# for speed in range(30,70,5):
#     speed = speed/10
speed = 3
for episode in range(TRAIN_EPISODES):
    env.reset()
    behavior_name=list(env.behavior_specs)[0]
    step = 0
    while True:
        index = random.randint(0, 3) # sample an action from action_dist
        action_onehot = F.one_hot(torch.tensor(index),num_actions).cpu()
        discrete_actions = np.array(action_onehot).reshape(1,4)*speed #[forward, backward, right, left]
        action_tuple = ActionTuple()
        action_tuple.add_discrete(discrete_actions)
        env.set_actions(behavior_name,action_tuple)
        env.step()
        step += 1

        decision_steps, terminal_steps = env.get_steps(behavior_name)
        if tracked_agent == -1 and len(decision_steps) >= 1:
            tracked_agent = decision_steps.agent_id[0]
            
        if tracked_agent in terminal_steps: # roll over or hit the target
            reward = terminal_steps[tracked_agent].reward
            if reward > 0: # hit the target
                print(f'{episode}: {step} in total')
                average += step
                break
            else:           # roll over or other conditions
                env.reset()
                step = 0
                continue # roll over or other unseen conditions
        if tracked_agent in decision_steps: # the agent which requires action
            continue
average /= TRAIN_EPISODES
print(f'For speed {speed}, average random step for hitting the target is {average}')
      

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Sample data
object_sizes = [1, 2, 3, 4, 5]  # Object sizes on the x-axis
max_steps = [[4721, 2468, 1587, 1249, 1084],   # Max steps for each object size
             [4027, 1550, 738, 640, 505],
             [3320, 1391, 464, 331, 307],
             [3289, 882, 679, 424, 211],
             [3057, 1203, 470, 378, 222]]
speeds = [1, 2, 3, 4, 5]  # Discrete speeds for color-coding

# Color mapping for each speed
speed_color_mapping = {
    1: 'red',
    2: 'blue',
    3: 'green',
    4: 'orange',
    5: 'purple'
}

# Generate scatter plot
for i, size in enumerate(object_sizes):
    for j, steps in enumerate(max_steps[i]):
        speed = speeds[j]
        color = speed_color_mapping[speed]
        plt.scatter(size, steps, c=color)

# Set labels and title
plt.xlabel('Object Size')
plt.ylabel('Max Step')
plt.title('Scatter Plot')

plt.xlim(0, 6)
plt.xticks(np.arange(0, 7, 1))
plt.ylim(0, 5000)
plt.yticks(np.arange(0, 5001, 200))

# Create legend
legend_labels = [f'Speed {speed}' for speed, _ in speed_color_mapping.items()]
plt.legend(legend_labels, loc='upper right')

# Display the plot
plt.show()


